In [ ]:
# External libraries
import numpy as np

# Drake dependencies
from pydrake.all import (
    DiagramBuilder,
    Simulator,
    StartMeshcat,
    MultibodyPlant,
    Demultiplexer,
    ConstantVectorSource,
    Parser,
    AddMultibodyPlantSceneGraph,
    ConstantVectorSource,
    DiagramBuilder,
    JointSliders,
    MeshcatVisualizer,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    Multiplexer,
    Parser,
    PrismaticJoint,
    SceneGraph,
    Simulator,
    SpatialInertia,
    Sphere,
    StartMeshcat,
    UnitInertia,
    MeshcatVisualizerParams,
    LoadModelDirectives,
    LoadModelDirectivesFromString,
    ProcessModelDirectives,
)

# Custom classes and functions
from CartesianStiffnessController import CartesianStiffnessController

# Helper functions
from manipulation.meshcat_utils import MeshcatSliders, StopButton
from manipulation.scenarios import AddShape
from manipulation.utils import RenderDiagram

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
model_directive = """
directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0.0]
        iiwa_joint_2: [0.6]
        iiwa_joint_3: [0.0]
        iiwa_joint_4: [-1.75]
        iiwa_joint_5: [0.0]
        iiwa_joint_6: [1.0]
        iiwa_joint_7: [0.0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://manipulation/schunk_wsg_50_welded_fingers.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
"""

In [ ]:
meshcat.DeleteAddedControls()

builder = DiagramBuilder()

# Add our iiwa to the scene
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
directives = LoadModelDirectivesFromString(model_directive)
parser = Parser(plant)
parser.package_map().Add("manipulation", "/home/trevor/.local/lib/python3.10/site-packages/manipulation/models/")
models = ProcessModelDirectives(directives, plant, parser)
iiwa = plant.GetModelInstanceByName("iiwa")
wsg = plant.GetModelInstanceByName("wsg")
# Not sure why calculated gravity compensation doesn't work. This simulates perfect gravity compensation.
plant.set_gravity_enabled(iiwa, False)
plant.set_gravity_enabled(wsg, False)
plant.Finalize()

# Connect meshcat to scene
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# Add our torque controller
controller: CartesianStiffnessController = builder.AddSystem(CartesianStiffnessController(plant, "iiwa"))
controller.SetGains(
    position=(10.0, 5.0),
    orientation=(10.0, 5.0),
    null_space=5.0
)

# Wire up controller torque to iiwa torque
builder.Connect(controller.GetOutputPort("iiwa_torque_cmd"), plant.get_actuation_input_port())

# Extract state from plant and feed to controller
state_demultiplexer = builder.AddSystem(Demultiplexer([7,7]))
builder.Connect(plant.get_state_output_port(iiwa), state_demultiplexer.get_input_port(0))
builder.Connect(state_demultiplexer.get_output_port(0), controller.GetInputPort("iiwa_position_measured"))
builder.Connect(state_demultiplexer.get_output_port(1), controller.GetInputPort("iiwa_velocity_measured"))

In [5]:
# Make a second plant for a desired point visualization
vis_scene_graph = builder.AddSystem(SceneGraph())
vis_plant = MultibodyPlant(time_step=0.005)
vis_plant.RegisterAsSourceForSceneGraph(vis_scene_graph)

# Add the point and a couple false bodies to give the point 3 DOF
point_body = AddShape(vis_plant, Sphere(0.03), "point_body", color=[0.9, 0.5, 0.5, 1.0])
vis_plant.AddRigidBody("false_body1", point_body, SpatialInertia(0, [0, 0, 0], UnitInertia(0, 0, 0)))
vis_plant.AddRigidBody("false_body2", point_body, SpatialInertia(0, [0, 0, 0], UnitInertia(0, 0, 0)))

# "Mount" the point on 3 prismatic joints
false_x = vis_plant.AddJoint(
    PrismaticJoint(
        "false_x",
        vis_plant.world_frame(),
        vis_plant.GetFrameByName("false_body1"),
        [1, 0, 0]
    ))
false_x.set_default_translation(0.47)
vis_plant.AddJointActuator("false_x", false_x)
false_y = vis_plant.AddJoint(
    PrismaticJoint(
        "false_y",
        vis_plant.GetFrameByName("false_body1"),
        vis_plant.GetFrameByName("false_body2"),
        [0, 1, 0]
    ))
vis_plant.AddJointActuator("false_y", false_y)
false_z = vis_plant.AddJoint(
    PrismaticJoint(
        "false_z",
        vis_plant.GetFrameByName("false_body2"),
        vis_plant.GetFrameByName("point_body"),
        [0, 0, 1]
    ))
false_z.set_default_translation(0.22)
vis_plant.AddJointActuator("false_z", false_z)
vis_plant.Finalize()

# Visualize the new scene graph
params = MeshcatVisualizerParams()
params.prefix = "teleop"
MeshcatVisualizer.AddToBuilder(
    builder,
    vis_scene_graph,
    meshcat,
    params
)

# Helper system to let us directly set the position of the point
positions_to_poses = builder.AddSystem(MultibodyPositionToGeometryPose(vis_plant))
builder.Connect(positions_to_poses.get_output_port(), vis_scene_graph.get_source_pose_port(vis_plant.get_source_id()))

# Position sliders for teleop control of the point
position_sliders = builder.AddSystem(
    JointSliders(
        meshcat,
        vis_plant,
        step=0.01,
        decrement_keycodes=["KeyA", "KeyS", "KeyZ"],
        increment_keycodes=["KeyD", "KeyW", "KeyX"],
    ))
builder.Connect(position_sliders.get_output_port(), positions_to_poses.get_input_port(),)

# Send the position of the point and 0 angle as the desired state for our end effector
positions_to_state = builder.AddSystem(Multiplexer([3, 3]))
zeros = builder.AddSystem(ConstantVectorSource([-1.362, 0, 1.57]))
builder.Connect(zeros.get_output_port(0), positions_to_state.get_input_port(0))
builder.Connect(position_sliders.get_output_port(), positions_to_state.get_input_port(1))
builder.Connect(positions_to_state.get_output_port(), controller.GetInputPort("pose_desired"))

# Stop button for ending sim
builder.AddSystem(StopButton(meshcat))

In [6]:
diagram = builder.Build()
simulator = Simulator(diagram)

In [7]:
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(np.inf)

Press Escape to stop the simulation
